In [1]:
import pandas as pd
import numpy as npa
import math
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import cohen_kappa_score
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',None)

In [2]:
abhishek_results = pd.read_csv('130_results_abhishek.csv').to_dict('records')
michel_results = pd.read_csv('130_results_michel.csv').to_dict('records') 

In [3]:
abhishek_results_map = {element['commit_hash']: element for element in abhishek_results }
michel_results_map = {element['commit_hash']: element for element in michel_results }

In [4]:
def sameResult(r1,r2):
    if r1 == r2: return True
    if r1 == 'none': 
        if r2 == 'unknown': return True
    if r1 == 'unknown': 
        if r2 == 'none': return True
    return False

In [18]:
categories = {
    "Timing and execution": 0,
    "Exchange of Information": 1,
    "Memory": 2
}

In [19]:
same_is_bug_fixing_commit = 0
is_bug_fixing_commit_conflict = []
both_is_bug_fixing_commit = 0

same_is_safety_related = 0
is_safety_related_conflict = []
both_is_safety_related = 0

same_type_of_safety_related = 0
type_of_safety_related_conflict = []

kappa_cohen_results = {
    'is_bug_fixing_commit': { 'Michel': [], 'Abhishek': [] },
    'is_safety_related': { 'Michel': [], 'Abhishek': [] },
    'type_of_safety_related': { 'Michel': [], 'Abhishek': [] }
}

for m_result in michel_results:
    a_result = abhishek_results_map[m_result['commit_hash']]

    kappa_cohen_results['is_bug_fixing_commit']['Michel'].append(1 if m_result['is_bug_fixing_commit'] else 0)
    kappa_cohen_results['is_bug_fixing_commit']['Abhishek'].append(1 if a_result['is_bug_fixing_commit'] else 0)
    
    # IS BUG-FIXING COMMIT
    if a_result['is_bug_fixing_commit'] == m_result['is_bug_fixing_commit']:
        same_is_bug_fixing_commit += 1
    else:
        is_bug_fixing_commit_conflict.append(
            (a_result['is_bug_fixing_commit'],a_result['comment'],m_result['is_bug_fixing_commit'], m_result['comment'])
        )

    # CASES WHERE BOTH AGREE ON "IS BUG-FIXING COMMIT"
    if a_result['is_bug_fixing_commit'] and m_result['is_bug_fixing_commit']:

        both_is_bug_fixing_commit+=1
        
        kappa_cohen_results['is_safety_related']['Michel'].append(
            1 if m_result['is_safety_related'] == 'true' else 0 if m_result['is_safety_related'] == 'false' else 2
        )
        kappa_cohen_results['is_safety_related']['Abhishek'].append(
            1 if a_result['is_safety_related'] == 'true' else 0 if a_result['is_safety_related'] == 'false' else 2
        )  
        
        # IS SAFETY-RELATED BUG
        if a_result['is_safety_related'] == m_result['is_safety_related']:
            same_is_safety_related += 1

            # CASES WHERE BOTH AGREE ON "SAFETY-RELATED BUG"
            if a_result['is_safety_related'] == 'true' and m_result['is_safety_related'] == 'true':
                both_is_safety_related += 1
                
                kappa_cohen_results['type_of_safety_related']['Michel'].append(categories[m_result['type_of_safety_related']])
                kappa_cohen_results['type_of_safety_related']['Abhishek'].append(categories[a_result['type_of_safety_related']])
                
                # SAFETY-RELATED BUG CATEGORY
                if a_result['type_of_safety_related'] == m_result['type_of_safety_related']:
                    same_type_of_safety_related += 1
                else:
                    type_of_safety_related_conflict.append(
                        (a_result['type_of_safety_related'], a_result['comment'], m_result['type_of_safety_related'], m_result['comment'])
                    )
        
        else:
            is_safety_related_conflict.append(
                (a_result['is_safety_related'], a_result['comment'], m_result['is_safety_related'], m_result['comment'])
            )

print("same_is_bug_fixing_commit: ", same_is_bug_fixing_commit, "/", len(michel_results), "(%.2f)"%(same_is_bug_fixing_commit*100/len(michel_results)))
print("same_is_safety_related: ", same_is_safety_related, "/", both_is_bug_fixing_commit, "(%.2f)"%(same_is_safety_related*100/both_is_bug_fixing_commit))
print("same_type_of_safety_related: ", same_type_of_safety_related, "/", both_is_safety_related, "(%.2f)"%(same_type_of_safety_related*100/both_is_safety_related))

same_is_bug_fixing_commit:  100 / 130 (76.92)
same_is_safety_related:  16 / 42 (38.10)
same_type_of_safety_related:  6 / 9 (66.67)


In [20]:
cohen_kappa_score(kappa_cohen_results['is_bug_fixing_commit']['Michel'], kappa_cohen_results['is_bug_fixing_commit']['Abhishek'])

0.5470383275261324

In [21]:
cohen_kappa_score(kappa_cohen_results['is_safety_related']['Michel'], kappa_cohen_results['is_safety_related']['Abhishek'])

0.12077294685990336

In [22]:
cohen_kappa_score(kappa_cohen_results['type_of_safety_related']['Michel'], kappa_cohen_results['type_of_safety_related']['Abhishek'])

0.47058823529411764

### Not same "Is a bug-fixing commit" reasoning

In [7]:
pd.DataFrame(is_bug_fixing_commit_conflict, columns =['Abhishek result', 'Abhishek comment', 'Michel result', 'Michel comment'])

,Abhishek result,Abhishek comment,Michel result,Michel comment
0,True,NaN,False,Only add sanity check
1,True,NaN,False,It seems to simply change how files/functions are displayed
2,True,NaN,False,Considered a fix but really just avoids a warning.
3,True,NaN,False,A condition is extended to cover only the necessary cases (and a section of code is not executed if it is not necessary).
4,True,NaN,False,There was a commented code and it is uncommented
5,True,NaN,False,It appears to fix a change introduced earlier; in this case to supplement a case not contemplated. I am not sure to what extent it is a fix
6,True,NaN,False,A condition is added so that when there are no interruptions; the code continues (saves time)
7,True,NaN,False,Does not look like a fix
8,True,NaN,False,A variable is renamed
9,True,NaN,False,Change oriented to debug


### Not same "Is a safety-related bug" reasoning

In [8]:
pd.DataFrame(is_safety_related_conflict, columns =['Abhishek result', 'Abhishek comment', 'Michel result', 'Michel comment'])

,Abhishek result,Abhishek comment,Michel result,Michel comment
0,true,Also I think it falls under the exchange of information category because the kernel-infoleak fix relates to loss or leak of information.,unknown,Looks like a fix but I'm not sure about the classification
1,true,NaN,unknown,Looks like a fix but not enough info
2,unknown,NaN,false,Check that is a BFC after checking mail list. A variable is not initialize but returned in some cases
3,false,NaN,unknown,Looks like a fix but no enough info
4,false,NaN,unknown,"Indicated as a ""Correct"" but i didnt understand the change"
5,true,Not 100% sure about the safety-type,unknown,There are too many changes in this merge. It seems to include some fixes but each one may be completely different
6,false,NaN,true,There appears to be an error forcing the system to exit. A flag is added to a command to avoid it
7,false,NaN,unknown,It looks like a fix but I'm not sure I understand what it fixes.
8,false,NaN,unknown,Seems to fix a performance problem
9,false,NaN,unknown,There are too many changes in this merge. It seems to include some fixes but each one may be completely different.


### Not same "Safety-related bug category" reasoning

In [9]:
pd.DataFrame(type_of_safety_related_conflict, columns =['Abhishek result', 'Abhishek comment', 'Michel result', 'Michel comment'])

,Abhishek result,Abhishek comment,Michel result,Michel comment
0,Timing and execution,NaN,Exchange of Information,Looks like a fix on bus ¿maybe a safety-relelated?
1,Exchange of Information,NaN,Timing and execution,Corrected a block of code that should be protected with a lock
2,Memory,NaN,Timing and execution,It is indicated that there is a co-occurrence problem if a section of code is not protected with locks.
